[배경] 
안녕하세요 여러분 :) 모델 튜닝 챌린지 : RF 하이퍼파라미터 최적화 경진대회에 오신 것을 환영합니다.

이번 챌린지에서는 랜덤포레스트(RF) 모델의 하이퍼파라미터를 최적화하여 최고의 성능을 달성하는 것을 목표로 합니다. 참가자들은 데이터 과학과 머신러닝의 중요한 측면인 모델 튜닝 기술을 연습하고, 실제 문제 해결에 적용할 수 있는 기회를 가질 것입니다.

또한 기존의 머신러닝 경진대회와는 다르게, 예측 결과를 제출하는 것이 아니라 최적의 하이퍼파라미터 세트를 제출하는 형식으로 진행됩니다. 이는 코딩 경험이 없는 초보자도 부담 없이 참가할 수 있는 '노 코딩' 대회로, 모든 데이커 여러분들이 머신러닝 모델의 성능을 극대화하는 방법을 배울 수 있는 독특한 기회를 제공합니다.

아울러, 아래의 가상 시나리오를 통해 본 해커톤의 배경과 목적을 보다 구체적으로 제시함으로써, 참가자들에게 실제 문제 해결 과정에 참여하고 있는 것처럼 느끼게 하여 동기부여를 높이고자 합니다.



<가상 시나리오>

데이콘에서 근무하고 있는 데이터 사이언티스트 'DACONIO'는 데이콘 유저들의 행동 데이터를 수집해 RF 모델로 유저의 다음 달 접속 여부를 예측하는 프로젝트에 몰두하고 있습니다. 
데이터 수집 및 정제를 마치고, 모델 학습을 위한 데이터 전처리까지 완료한 'DACONIO'는 이제 최적의 RF 모델을 구축하기 위한 하이퍼파라미터 튜닝 단계에 도달했습니다. 
이 중요한 단계를 해결하기 위해, 'DACONIO'는 데이콘 커뮤니티의 지혜를 모으고자 합니다. 여러분의 도전은 데이콘 유저들의 다음 달 접속 여부를 더 정확하게 예측할 수 있는 최적화된 RF 모델의 하이퍼파라미터를 찾는 것입니다.


이번 챌린지를 통해 랜덤포레스트 모델의 내부 작동 방식을 더 깊이 이해하고, 실제 세계 데이터 과학 문제에 적용하는 데 필요한 튜닝 기술을 개발할 수 있을 것입니다. 

데이커 여러분들이 이 도전을 통해 성장하고, 새로운 지식을 쌓으며, 데이콘 커뮤니티 내에서 교류하게 되기를 기대합니다.



[주제]
RF(랜덤포레스트) 하이퍼파라미터 최적화



[설명]
RF(랜덤포레스트) 모델이 최상의 성능을 달성할 수 있는 최적의 하이퍼파라미터를 탐색



본 대회는 RF 모델을 튜닝할 하이퍼파라미터를 제출 양식에 맞게 제출해야하며, 제출 시 자동적으로 평가 서버에서 학습 데이터로 RF 모델을 학습하고 숨겨진 평가 데이터로부터 추론 후 채점이 이루어집니다.

평가 서버에서의 RF 모델을 학습하고 추론하는 환경은 아래와 같습니다.

OS : Ubuntu 18.04.3 LTS
Python : 3.6.9
Sklearn : 0.21.3
Random Seed : 42


[주최 / 주관]
데이콘



[참가 대상]
데이커라면 누구나

1. 리더보드
평가 산식 : AUC (Area Under the ROC Curve)
Public Score: 전체 테스트 샘플 중 사전 샘플링된 49%
Private Score: 전체 테스트 샘플 중 나머지 51%


2. 평가
최종 평가: 리더보드 Private Score


3. 참여
개인으로만 참여할 수 있습니다.
참가 등록 시 설정한 대학 소속과 닉네임이 리더보드에 표시됩니다.
개인 참가 방법: 팀 신청 없이, 자유롭게 제출 창에서 제출 가능
1일 제출 가능 횟수: 5회


4. 유의 사항
1일 최대 제출 횟수: 5회
사용 가능 언어: 제한 없음
본 경진대회에 한하여 수기로 작성된 하이퍼 파라미터 제출도 허용합니다.
최종 순위는 선택된 파일 중에서 채점되므로 참가자는 제출 창에서 자신이 최종적으로 채점 받고 싶은 파일 1개를 선택해야 함
데이콘은 부정 제출 행위를 엄격히 금지하고 있으며, 데이콘 대회 부정 제출 이력이 있는 경우 평가가 제한됩니다. 자세한 사항은 아래의 링크를 참고해 주시기 바랍니다. https://dacon.io/notice/notice/13


7. 문의
대회 운영 및 데이터 이상에 관련된 질문 외에는 답변을 드리지 않고 있습니다. 기타 질문은 토론 페이지를 통해 자유롭게 토론해 주시기 바랍니다.
데이콘 답변을 희망하는 경우 [토크] 페이지 대회 문의 게시글에 댓글을 올려 주시기 바랍니다.

[세부일정]
- 대회 기간 : 2024년 03월 11일 10:00 ~ 2024년 04월 08일 10:00

- 대회 종료 : 2024년 04월 08일 10:00

- 최종 수상자 발표 : 2024년 04월 08일 10:00



※ 세부 일정은 대회 운영 상황에 따라 변동될 수 있습니다